# Thomas Fire perimeter data retrieval and selection

Author: Marina Kochuten

Repository: https://github.com/marinakochuten/eds220-hwk4

## About

The Thomas Fire, which burned over 280,000 acres in Ventura and Santa Barbara counties in December 2017, was one of California’s largest wildfires at the time. It caused widespread ecological damage, displaced communities, and left lasting environmental impacts.

Using open-access data containing fire perimeters in California, I create a geospatial file containing the boundary of the 2017 Thomas Fire.

### Highlights

- Accessing open-access data online from a reputable source
- Exploring and filtering geospatial data with `geopandas`
- Creating geospatial files from `geopandas.GeoDataFrame` using `to_file()` method

### Data

**About the data:** I use California Fire Perimeter data from the State of California's Data Catalog. The dataset is updated annually and includes fire perimeters dating back to 1878.

**Citation:** State of California Data Catalog (2024), *California Fire Perimeters (all)* [Data set] Available from: https://catalog.data.gov/dataset/california-fire-perimeters-all-b3436. Access date: November 18, 2024.

## Import libraries

In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
import os

In [2]:
# Set pandas to display all columns
pd.set_option('display.max_columns', None)

## Data exploration

In [3]:
"""
Set environment variable 'PROJ_LIB' to specify the path for the PROJ library
Resolves ERROR 1: PROJ: proj_create_from_database
"""""
os.environ['PROJ_LIB'] = '/opt/anaconda3/share/proj'

# Import california fire perimeter data
ca_fires = gpd.read_file('data/california_fire_perimeters/california_fire_perimeters.shp')

In [4]:
# Look at the first few rows of the data
ca_fires.head(3)

,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,C_METHOD,OBJECTIVE,GIS_ACRES,COMMENTS,COMPLEX_NA,IRWINID,FIRE_NUM,COMPLEX_ID,DECADES,geometry
0,2023,CA,CDF,SKU,WHITWORTH,00004808,2023-06-17,2023-06-17,5,1,1,5.72913,None,None,{7985848C-0AC2-4BA4-8F0E-29F778652E61},None,None,2020,"POLYGON ((-13682443.000 5091132.739, -13682445..."
1,2023,CA,LRA,BTU,KAISER,00010225,2023-06-02,2023-06-02,5,1,1,13.60240,None,None,{43EBCC88-B3AC-48EB-8EF5-417FE0939CCF},None,None,2020,"POLYGON ((-13576727.142 4841226.161, -13576726..."
2,2023,CA,CDF,AEU,JACKSON,00017640,2023-07-01,2023-07-02,2,1,1,27.81450,None,None,{B64E1355-BF1D-441A-95D0-BC1FBB93483B},None,None,2020,"POLYGON ((-13459243.000 4621236.000, -13458968..."


In [5]:
# Find the time range for the data
ca_fires['YEAR_'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002,
       2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991,
       1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980,
       1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969,
       1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958,
       1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948, 1947,
       1946, 1945, 1944, 1943, 1942, 1941, 1940, 1939, 1938, 1937, 1936,
       1935, 1934, 1933, 1932, 1931, 1930, 1929, 1928, 1927, 1926, 1925,
       1924, 1923, 1922, 1921, 1920, 1919, 1918, 1917, 1916, 1915, 1914,
       1913, 1912, 1911, 1910, 1909, 1908, 1907, 1906, 1905, 1903, 1902,
       1900, 1898, 1896, 1895, 1878,    0])

In [6]:
# Check the coordinate reference system
ca_fires.crs

<Projected CRS: PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GE ...>
Name: WGS_1984_Web_Mercator_Auxiliary_Sphere
Axis Info [cartesian]:
- [east]: Easting (Meter)
- [north]: Northing (Meter)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
# Is it geographic or projected?
print('is geographic? ', ca_fires.crs.is_geographic)
print('is projected? ', ca_fires.crs.is_projected)

is geographic?  False
is projected?  True


**Preliminary summary:** The California fire perimeter data contains data from the around the late 1800s to 2023. It is in the projeted CRS WGS 84 / Pseudo-Mercator (EPSG:3857). The columns `YEAR_` and `FIRE_NAME` will be useful for subsetting to the 2017 Thomas Fire. 

## Prepare Thomas Fire boundary

In [8]:
# Update column names to lower case
ca_fires.columns = ca_fires.columns.str.lower()
ca_fires.columns

Index(['year_', 'state', 'agency', 'unit_id', 'fire_name', 'inc_num',
       'alarm_date', 'cont_date', 'cause', 'c_method', 'objective',
       'gis_acres', 'comments', 'complex_na', 'irwinid', 'fire_num',
       'complex_id', 'decades', 'geometry'],
      dtype='object')

In [9]:
# Select the 2017 Thomas Fire boundary
thomas_fire = ca_fires[(ca_fires.fire_name == "THOMAS") & (ca_fires.year_ == 2017)]
thomas_fire

,year_,state,agency,unit_id,fire_name,inc_num,alarm_date,cont_date,cause,c_method,objective,gis_acres,comments,complex_na,irwinid,fire_num,complex_id,decades,geometry
2654,2017,CA,USF,VNC,THOMAS,00003583,2017-12-04,2018-01-12,9,7,1,281791.0,CONT_DATE based on Inciweb,None,None,None,None,2010,"MULTIPOLYGON (((-13316089.016 4088553.040, -13..."


In [10]:
# Save Thomas Fire boundary as a geospatial file
thomas_fire.to_file("data/thomas_fire_boundary.geojson", crs = 'epsg:3857')  # CRS from ca_fires data

I saved the Thomas Fire boundary as a GeoJSON, because they tend to be smaller in size than shapefiles. I will now use this file to visualize the Thomas Fire burn scar, in the next notebook in this repository: hwk4-task2-false-color-kochuten.ipynb.